<a href="https://colab.research.google.com/github/ABCutter/Cohort-18-Projects/blob/main/Project_3_SQL_2_BQ_project_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project SQL

## Pick a dataset that interests you (or multiple data sets)

Use the Open Data Sets available from Google BigQuery. You can use your own Google account or Kaggle.



## Come up with questions about your data
* What sort of information is in this dataset?
* How many records are there?
* Have the number of bitcoin transactions increased year over year?
* Does New Mexico get more or less rain now than 20 years ago?
* How many different countries (states, counties, cities, etc) have records in this data set?




## Use SQL queries to pull specific information

Do NOT pull all the data and then filter using DataFrame methods etc. Make sure and use AT LEAST 13 of the 15 SQL options listed below. (You may have to get creative and come up with more questions to ask/answer.)

In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.cm as cm
import seaborn as sns

## Data Collection/Sources


In [ ]:
##SETUP

# Step 1: Authenticate
from google.colab import auth
auth.authenticate_user()

# Step 2: Import packages
from google.cloud import bigquery

# Step 3: Set project ID
project_id = "abc-ddds-p3-cdc"
client = bigquery.Client(project=project_id)

from google.colab import userdata

billing_project_id=userdata.get('bq_billing_project_id')
billing_project_id

In [ ]:
%%capture
%%bash
apt-get update
apt-get install -y sqlite3

In [ ]:
!sqlite3 --help
#Check to make sure it's running

### Basic Queries


In [ ]:
!bq query --project_id=abc-ddds-p3-cdc --nouse_legacy_sql \
'SELECT COUNT(*) FROM `bigquery-public-data.usa_names.usa_1910_2013`'


In [ ]:
query = """
SELECT *
FROM `bigquery-public-data.usa_names.usa_1910_2013`
WHERE state = 'NM'
AND year BETWEEN 2000 AND 2013
LIMIT 10000
"""

df = client.query(query).to_dataframe()
df


## Data Cleaning


In [ ]:
df.isnull().sum().sum()

In [ ]:
df.describe()

No data imputation is necessary

## Exploratory Data Analysis


Use the following: SELECT, WHERE, AND, OR, LIKE, BETWEEN, LIMIT

In [ ]:
#Asked ChatGPT about loading the data, and this was recommended, so I went with it
# Create SQLite in-memory database and write the DataFrame to a table
conn = sqlite3.connect(':memory:')
df.to_sql('names', conn, index=False, if_exists='replace')

# Example SQLite3 query
query_sqlite = """
SELECT name, SUM(number) AS total
FROM names
GROUP BY name
ORDER BY total DESC
LIMIT 100;
"""

pd.read_sql(query_sqlite, conn)


In [ ]:
# Female baby names in NM between 2000 and 2013
query_1 = """
SELECT name, gender, year, number
FROM `bigquery-public-data.usa_names.usa_1910_2013`
WHERE state = 'NM'
  AND gender = 'F'
  AND year BETWEEN 2000 AND 2013
LIMIT 100;
"""

df_1 = client.query(query_1).to_dataframe()
df_1


In [ ]:
# Names in 2010 containing 'Ann' or 'Marie'
query_2 = """
SELECT name, year, number
FROM `bigquery-public-data.usa_names.usa_1910_2013`
WHERE state = 'NM'
  AND year = 2010
  AND (name LIKE '%An%' OR name LIKE '%Mar%')
LIMIT 100;
"""

df_2 = client.query(query_2).to_dataframe()
df_2


In [ ]:
# Names Ava or Lia% in 2008
query_3 = """
SELECT name, gender, number
FROM `bigquery-public-data.usa_names.usa_1910_2013`
WHERE state = 'NM'
  AND year = 2009
  AND (name LIKE 'Av%' OR name LIKE 'Lia%')
LIMIT 10;
"""

df_3 = client.query(query_3).to_dataframe()
df_3


In [ ]:
# Top names in NM between 2005 and 2010
query_4 = """
SELECT name, SUM(number) AS total
FROM `bigquery-public-data.usa_names.usa_1910_2013`
WHERE state = 'NM'
  AND year BETWEEN 2005 AND 2010
GROUP BY name
ORDER BY total DESC
LIMIT 100;
"""

df_4 = client.query(query_4).to_dataframe()
df_4


### Sorting and Grouping


Use the following: ORDER BY, DISCTINCT, GROUP BY

In [ ]:
# Top 10 female names in NM between 2000 and 2013
query_5 = """
SELECT name, SUM(number) AS total
FROM `bigquery-public-data.usa_names.usa_1910_2013`
WHERE state = 'NM'
  AND gender = 'F'
  AND year BETWEEN 2000 AND 2013
GROUP BY name
ORDER BY total DESC
LIMIT 10;
"""

df_5 = client.query(query_5).to_dataframe()
df_5


In [ ]:
# Most popular names in 2010 in NM
query_6 = """
SELECT name, SUM(number) AS total
FROM `bigquery-public-data.usa_names.usa_1910_2013`
WHERE state = 'NM'
  AND year = 2010
GROUP BY name
ORDER BY total DESC
LIMIT 10;
"""

df_6 = client.query(query_6).to_dataframe()
df_6


In [ ]:
# All states included in the dataset
query_7 = """
SELECT DISTINCT state
FROM `bigquery-public-data.usa_names.usa_1910_2013`
ORDER BY state;
"""

df_7 = client.query(query_7).to_dataframe()
df_7


In [ ]:
# Number of years each gendered name appears in NM
query_8 = """
SELECT name, gender, COUNT(DISTINCT year) AS active_years
FROM `bigquery-public-data.usa_names.usa_1910_2013`
WHERE state = 'NM'
GROUP BY name, gender
ORDER BY active_years DESC
LIMIT 100;
"""

df_8 = client.query(query_8).to_dataframe()
df_8


## Processing


### Aggregates


Use the following: MAX, MIN, SUM, AVG, COUNT

In [ ]:
# Average, max, and min number of babies per name
query_9 = """
SELECT name,
       ROUND(AVG(number)) AS avg_per_year,
       MAX(number) AS max_in_year,
       MIN(number) AS min_in_year
FROM `bigquery-public-data.usa_names.usa_1910_2013`
WHERE state = 'NM'
  AND year BETWEEN 1950 AND 2013
GROUP BY name
ORDER BY avg_per_year DESC
LIMIT 10;
"""

df_9 = client.query(query_9).to_dataframe()
df_9


In [ ]:
# Count how many unique names were used each year
query_10 = """
SELECT year, COUNT(DISTINCT name) AS unique_names
FROM `bigquery-public-data.usa_names.usa_1910_2013`
WHERE state = 'NM'
GROUP BY year
ORDER BY year;
"""

df_10 = client.query(query_10).to_dataframe()
df_10


In [ ]:
# Total babies born each year in NM
query_11 = """
SELECT year, SUM(number) AS total_births
FROM `bigquery-public-data.usa_names.usa_1910_2013`
WHERE state = 'NM'
GROUP BY year
ORDER BY year;
"""

df_11 = client.query(query_11).to_dataframe()
df_11


In [ ]:
# Highest single-year popularity of any name in 2012
query_12 = """
SELECT name, number
FROM `bigquery-public-data.usa_names.usa_1910_2013`
WHERE state = 'NM'
  AND year = 2013
ORDER BY number DESC
LIMIT 10;
"""

df_12 = client.query(query_12).to_dataframe()
df_12


In [ ]:
# Minimum number of babies per name per year in NM
# Find the min number of all years (5)
query_14 = """
SELECT year, MIN(number) AS min_per_year
FROM `bigquery-public-data.usa_names.usa_1910_2013`
WHERE state = 'NM'
GROUP BY year
ORDER BY year;
"""

df_14 = client.query(query_14).to_dataframe()
df_14.min()

The lowest name count in this dataset is consistently 5, across all years. This is because the U.S. Social Security Administration only reports names with 5 or more occurrences per year, to (statistically) protect individual privacy.

In [ ]:
# # Find years where at least one name was used only once #THIS ENDED UP NOT BEING RELEVANT
# query_15 = """
# SELECT DISTINCT year
# FROM `bigquery-public-data.usa_names.usa_1910_2013`
# WHERE state = 'NM' AND number = 5
# ORDER BY year;
# """

# df_15 = client.query(query_15).to_dataframe()
# df_15


In [ ]:
# # Names used only once in a known-valid year (e.g., 2004) #THIS ENDED UP NOT BEING RELEVANT
# query_16 = """
# SELECT name, number
# FROM `bigquery-public-data.usa_names.usa_1910_2013`
# WHERE state = 'NM'
#   AND year = 2004
#   AND number = 5
# LIMIT 100;
# """

# df_16 = client.query(query_16).to_dataframe()
# df_16


In [ ]:
# Most popular names from 2000–2013 (total summed across years)
query_17 = """
SELECT name, SUM(number) AS total
FROM `bigquery-public-data.usa_names.usa_1910_2013`
WHERE state = 'NM' AND year BETWEEN 2000 AND 2013
GROUP BY name
ORDER BY total DESC
LIMIT 10;
"""

df_17 = client.query(query_17).to_dataframe()
df_17


## Data Visualization/Communication of Results


## Make some plots

Make some cool plots to go with your data. Write SQL queries to get ONLY the information you need for each plot. (Don't pull ALL the data and then just plot a few columns.)



In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(df_11['year'], df_11['total_births'], color='green', marker='o')
plt.title('Total Recorded Baby Names per Year in New Mexico')
plt.xlabel('Year')
plt.ylabel('Total Births (Name Records ≥ 5)')
plt.grid(True)
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(df_10['year'], df_10['unique_names'], color='purple', marker='o')
plt.title('Unique Baby Names per Year in New Mexico')
plt.xlabel('Year')
plt.ylabel('Number of Unique Names (Name Records ≥ 5)')
plt.grid(True)
plt.show()


In [ ]:
# Bar chart for top 10 female names 2000–2013
plt.figure(figsize=(10, 6))
plt.bar(df_5['name'], df_5['total'], color='turquoise')
plt.title('Top 10 Girl Names in New Mexico (2000–2013)')
plt.xlabel('Name')
plt.ylabel('Total Babies Named')
plt.xticks(rotation=60)
plt.grid(True)
plt.show()


In [ ]:
query_isabella = """
SELECT year, SUM(number) AS total
FROM `bigquery-public-data.usa_names.usa_1910_2013`
WHERE state = 'NM' AND name = 'Isabella'
GROUP BY year
ORDER BY year;
"""
df_isabella = client.query(query_isabella).to_dataframe()


In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(df_isabella['year'], df_isabella['total'], marker='o')
plt.title('Popularity of "Isabella" in NM Over Time')
plt.xlabel('Year')
plt.ylabel('Babies Named Isabella')
plt.grid(True)
plt.show()


In [ ]:
# Get the top 5 female baby names in NM (2000–2013)
query_top5_names = """
SELECT name, SUM(number) AS total
FROM `bigquery-public-data.usa_names.usa_1910_2013`
WHERE state = 'NM' AND gender = 'F' AND year BETWEEN 2000 AND 2013
GROUP BY name
ORDER BY total DESC
LIMIT 5;
"""
df_top5 = client.query(query_top5_names).to_dataframe()
top5_names = df_top5['name'].tolist()
top5_names


In [ ]:
# Query 2000-2013 for the top 5 names
query_top5_trend = f"""
SELECT name, year, SUM(number) AS total
FROM `bigquery-public-data.usa_names.usa_1910_2013`
WHERE state = 'NM'
  AND gender = 'F'
  AND year BETWEEN 2000 AND 2013
  AND name IN ({', '.join([f"'{name}'" for name in top5_names])})
GROUP BY name, year
ORDER BY name, year;
"""

df_top5_trend = client.query(query_top5_trend).to_dataframe()
df_top5_trend.head()


In [ ]:
plt.figure(figsize=(10, 6))

# Plot one line per name
for name in top5_names:
    df_subset = df_top5_trend[df_top5_trend['name'] == name]
    plt.plot(df_subset['year'], df_subset['total'], marker='o', label=name)

plt.title('Top 5 Female Baby Names in New Mexico (2000–2013)')
plt.xlabel('Year')
plt.ylabel('Number of Babies')
plt.legend(title="Name")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# Get the top 5 boy names in NM (2000–2013)
query_top5_boys = """
SELECT name, SUM(number) AS total
FROM `bigquery-public-data.usa_names.usa_1910_2013`
WHERE state = 'NM' AND gender = 'M' AND year BETWEEN 2000 AND 2013
GROUP BY name
ORDER BY total DESC
LIMIT 5;
"""
df_top5_boys = client.query(query_top5_boys).to_dataframe()
top5_boy_names = df_top5_boys['name'].tolist()
top5_boy_names


In [ ]:
# Query 2000–2013 for the top 5 male names
query_top5_boy_trend = f"""
SELECT name, year, SUM(number) AS total
FROM `bigquery-public-data.usa_names.usa_1910_2013`
WHERE state = 'NM'
  AND gender = 'M'
  AND year BETWEEN 2000 AND 2013
  AND name IN ({', '.join([f"'{name}'" for name in top5_boy_names])})
GROUP BY name, year
ORDER BY name, year;
"""

df_top5_boy_trend = client.query(query_top5_boy_trend).to_dataframe()
df_top5_boy_trend.head()


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))

# Plot one line per name
for name in top5_boy_names:
    df_subset = df_top5_boy_trend[df_top5_boy_trend['name'] == name]
    plt.plot(df_subset['year'], df_subset['total'], marker='o', label=name)

plt.title('Top 5 Male Baby Names in New Mexico (2000–2013)')
plt.xlabel('Year')
plt.ylabel('Number of Babies')
plt.legend(title="Name")
plt.grid(True)
plt.tight_layout()
plt.show()


## EXTRA CREDIT:

#### Use a query that joins two tables.


#### Make a model to see if you can predict something


#### Come up with something else cool to do with your data
